# TensorFlow BYOM:
## カスタムトレーニングスクリプトでトレーニングし、Neoでコンパイルし、SageMakerで展開

このノートブックは、[TensorFlow MNIST distributed training notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_distributed_mnist/tensorflow_distributed_mnist.ipynb) の拡張版で、Amazon SageMaker Neoを用いた例です。同じ分類タスクを実行しますが、今回はNeo APIバックエンドを使用して訓練モデルをコンパイルし、ハードウェアの選択に最適化します。 最後に、Neo Deep Learning Runtimeを使用してコンパイルされたモデルのために、SageMakerにリアルタイムでホストされたエンドポイントを設定しています。

### 環境の設定

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

### MNIST datasetのダウンロード

In [2]:
import utils
from tensorflow.contrib.learn.python.learn.datasets import mnist
import tensorflow as tf

data_sets = mnist.read_data_sets('data', dtype=tf.uint8, reshape=False, validation_size=5000)

utils.convert_to(data_sets.train, 'train', 'data')
utils.convert_to(data_sets.validation, 'validation', 'data')
utils.convert_to(data_sets.test, 'test', 'data')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.


Extracting data/train-images-idx3-ubyte.gz


Instructions for updating:
Please use tf.data to implement this functionality.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Writing data/train.tfrecords
Writing data/validation.tfrecords
Writing data/test.tfrecords


## データのアップロード
```sagemaker.Session.upload_data```関数を使ってデータセットをS3の場所にアップロードします。
戻り値の`inputs`は、トレーニングジョブを開始するときに使用します。

In [3]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-mnist')

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-314676777416


## 分散トレーニング用のスクリプトを作成する
エントリポイントとして使用するネットワークモデルのコードは次のとおりです。

In [4]:
!pygmentize 'mnist.py'

import os
import tensorflow as tf
from tensorflow.python.estimator.model_fn import ModeKeys as Modes

INPUT_TENSOR_NAME = 'inputs'
SIGNATURE_NAME = 'predictions'

LEARNING_RATE = 0.001


def model_fn(features, labels, mode, params):
    # Input Layer
    input_layer = tf.reshape(features[INPUT_TENSOR_NAME], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.d

このスクリプトは、[TensorFlow MNIST example](https://github.com/tensorflow/models/tree/master/official/mnist)です。 
`` model_fn（機能、ラベル、モード） ``を提供します。これは、学習、評価、推論に使用されます。 トレーニングスクリプトの詳細については、[TensorFlow MNIST distributed training notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_distributed_mnist/tensorflow_distributed_mnist.ipynb) を参照してください。

トレーニングスクリプトの最後には、Neo Deep Learning Runtimeで使用する2つの追加機能があります。
* `neo_preprocess（payload、content_type）`：各受信リクエストのペイロードとContent-Typeを受け取り、NumPy配列を返す関数
* `neo_postprocess（result）`：Deep Learining Runtimeによって生成された予測結果を受け取り、応答本体を返す関数

## sagemaker.TensorFlow estimatorを使用してトレーニングジョブを作成する

In [5]:
from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             framework_version='1.11.0',
                             training_steps=1000, 
                             evaluation_steps=100,
                             train_instance_count=2,
                             train_instance_type='ml.c5.2xlarge')

%time mnist_estimator.fit(inputs)

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-314676777416
INFO:sagemaker:Creating training-job with name: sagemaker-tensorflow-2018-12-17-17-52-05-942


2018-12-17 17:52:06 Starting - Starting the training job...
2018-12-17 17:52:07 Starting - Launching requested ML instances......
2018-12-17 17:53:09 Starting - Preparing the instances for training.....
2018-12-17 17:54:19,112 INFO - root - running container entrypoint
2018-12-17 17:54:19,112 INFO - root - starting train task
2018-12-17 17:54:19,125 INFO - container_support.training - Training starting
2018-12-17 17:54:21,078 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2018-12-17 17:54:21,078 INFO - tf_container - {"environment": "cloud", "cluster": {"worker": ["algo-2:2222"], "ps": ["algo-1:2223", "algo-2:2223"], "master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
2018-12-17 17:54:21,078 INFO - tf_container - ---------------------------------------------------------
2018-12-17 17:54:21,078 INFO - tf_container - creating RunConfig:
2018-12-17 17:54:21,078 INFO - tf_container - {'save_checkpoints_secs': 300}
2018-12-17 17:54:21,078 INF

2018-12-17 17:54:38,084 INFO - tensorflow - global_step/sec: 15.3203
2018-12-17 17:54:41,695 INFO - tensorflow - loss = 0.042283025, step = 179 (12.463 sec)
2018-12-17 17:54:44,422 INFO - tensorflow - loss = 0.029699039, step = 220 (13.125 sec)
2018-12-17 17:54:44,804 INFO - tensorflow - global_step/sec: 15.1804
2018-12-17 17:54:51,253 INFO - tensorflow - global_step/sec: 15.6614
2018-12-17 17:54:55,239 INFO - tensorflow - loss = 0.0490227, step = 387 (13.544 sec)
2018-12-17 17:54:56,679 INFO - tensorflow - loss = 0.045940056, step = 411 (12.256 sec)
2018-12-17 17:54:57,825 INFO - tensorflow - global_step/sec: 15.52
2018-12-17 17:55:04,156 INFO - tensorflow - global_step/sec: 15.7951
2018-12-17 17:55:08,787 INFO - tensorflow - loss = 0.056856263, step = 600 (13.548 sec)
2018-12-17 17:55:08,691 INFO - tensorflow - loss = 0.06102873, step = 599 (12.012 sec)
2018-12-17 17:55:10,656 INFO - tensorflow - global_step/sec: 15.5388
2018-12-17 17:55:17,076 INFO - tensorflow - global_step/sec: 15


2018-12-17 17:55:55 Uploading - Uploading generated training model
2018-12-17 17:55:55 Completed - Training job completed
Billable seconds: 211
CPU times: user 425 ms, sys: 52.8 ms, total: 478 ms
Wall time: 4min 13s


The **```fit```** method will create a training job in two **ml.c5.2xlarge** instances. The logs above will show the instances doing training, evaluation, and incrementing the number of **training steps**. 

In the end of the training, the training job will generate a saved model for TF serving.

**```fit```** メソッドは、2つの **ml.c5.2xlarge** インスタンスでトレーニングジョブを作成します。 上記のログには、トレーニング、評価、**トレーニングステップ数の増分** などのインスタンスが表示されます。

訓練の最後に、訓練の仕事は、TF提供のために保存されたモデルを生成するでしょう。

# Deploy the trained model to prepare for predictions (the old way)

The deploy() method creates an endpoint which serves prediction requests in real-time.

### 訓練されたモデルを展開して予測を準備する（既存の手法）

deploy（）メソッドは、予測リクエストをリアルタイムで処理するエンドポイントを作成します。

In [ ]:
mnist_predictor = mnist_estimator.deploy(initial_instance_count=1,
                                         instance_type='ml.c5.4xlarge')

## エンドポイントの呼び出し

In [ ]:
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

for i in range(10):
    data = mnist.test.images[i].tolist()
    tensor_proto = tf.make_tensor_proto(values=np.asarray(data), shape=[1, len(data)], dtype=tf.float32)
    predict_response = mnist_predictor.predict(tensor_proto)
    
    print("========================================")
    label = np.argmax(mnist.test.labels[i])
    print("label is {}".format(label))
    prediction = predict_response['outputs']['classes']['int64_val'][0]
    print("prediction is {}".format(prediction))

## エンドポイントの削除

In [ ]:
sagemaker.Session().delete_endpoint(mnist_predictor.endpoint)

# Deploy the trained model using Neo

Now the model is ready to be compiled by Neo to be optimized for our hardware of choice. We are using the  ``TensorFlowEstimator.compile_model`` method to do this. For this example, our target hardware is ``'ml_c5'``. You can changed these to other supported target hardware if you prefer.

## Compiling the model
The ``input_shape`` is the definition for the model's input tensor and ``output_path`` is where the compiled model will be stored in S3. **Important. If the following command result in a permission error, scroll up and locate the value of execution role returned by `get_execution_role()`. The role must have access to the S3 bucket specified in ``output_path``.**

## Neoを使用して訓練されたモデルを展開する

これでモデルはNeoによってコンパイルされ、選択したハードウェアに最適化されました。 これを行うには `` TensorFlowEstimator.compile_model``メソッドを使用しています。 この例では、ターゲットハードウェアは `` 'ml_c5'``です。 必要に応じて、これらを他のサポートされているターゲットハードウェアに変更できます。

### モデルをコンパイルする
`` input_shape``はモデルの入力テンソルの定義で、 `` output_path``はコンパイルされたモデルがS3に格納される場所です。 **重要。 次のコマンドでパーミッションエラーが発生した場合は、スクロールして `get_execution_role（）`によって返された実行ロールの値を探します。 ロールは `` output_path``で指定されたS3バケットにアクセスする必要があります。**

In [ ]:
output_path = '/'.join(mnist_estimator.output_path.split('/')[:-1])
optimized_estimator = mnist_estimator.compile_model(target_instance_family='ml_c5', 
                              input_shape={'data':[1, 784]},  # Batch size 1, 3 channels, 224x224 Images.
                              output_path=output_path,
                              framework='tensorflow', framework_version='1.11.0')

## コンパイルされたモデルのデプロイ

In [ ]:
optimized_predictor = optimized_estimator.deploy(initial_instance_count = 1,
                                                 instance_type = 'ml.c5.4xlarge')

In [ ]:
# The neo_preprocess() function expects an image in the request body
# But the MNIST example data is saved as NumPy array.
# So we convert it to PNG before invoking the endpoint
def png_serializer(data):
    im = PIL.Image.fromarray(data.reshape((28,28))*255).convert('L')
    f = io.BytesIO()
    im.save(f, format='png')
    f.seek(0)
    return f.read()

optimized_predictor.content_type = 'application/x-image'
optimized_predictor.serializer = png_serializer

## エンドポイントの読み出し

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
from IPython import display
import PIL.Image
import io

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

for i in range(10):
    data = mnist.test.images[i]
    # Display image
    im = PIL.Image.fromarray(data.reshape((28,28))*255).convert('L')
    display.display(im)
    # Invoke endpoint with image
    predict_response = optimized_predictor.predict(data)
    
    print("========================================")
    label = np.argmax(mnist.test.labels[i])
    print("label is {}".format(label))
    prediction = predict_response
    print("prediction is {}".format(prediction))

## エンドポイントの削除

In [ ]:
sagemaker.Session().delete_endpoint(optimized_predictor.endpoint)